<img src="Bilder\ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN01/1.1-Einführung_NN-EDA.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
%pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt

In [ ]:
# Imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
import torch
from IPython.display import Image, display
from matplotlib.colors import ListedColormap
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.utilities.model_summary import ModelSummary
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron
from torch import nn
from torch.utils.data import DataLoader
from torchmetrics.classification import Accuracy
from torchsummary import summary
from torchvision import datasets, transforms

# Machine Learnining und Deep Learning

Quellen: 
- **Aurélien Géron:** *Praxiseinstieg Machine Learning mit Scikit-Learn und Tensorflow*: Konzepte, Tools und Techniken für intelligenze Systeme, Authorized German translation of the English edition of Hands-On Machine Learning with Scikit-Learn and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems, ISBN 978-1-491-96229-9
- **Francois Chollet**: *Deep Learning mit Python und Keras*, Das Praxis-HandbuchISBN 978-3-95845-839-0, 1. Auflage 2018, www.mitp.de, © 2018 mitp Verlags GmbH & Co. KG, Frechen, Übersetzung der amerikanischen Originalausgabe François Chollet: Deep Learning with Python, ISBN 978-1617294433


- Machine Learning entstand aufgrund folgender Frage: Könnte ein Computer über das, »von dem wir wissen, wie wir es befehlen können«, hinausgehen und selbst erlernen, wie eine bestimmte Aufgabe erledigt wird? Könnte ein Computer uns überraschen? Könnte ein Computer automatisch Regeln erlernen, indem er Daten betrachtet, ohne dass Programmierer diese Datenverarbeitungsregeln von Hand erstellen müssen?
- Diese Fragen öffneten einem neuen Programmierparadigma Tür und Tor. Bei der klassischen Programmierung, der symbolischen KI, geben Menschen Regeln (ein Programm) und die gemäss diesen Regeln zu verarbeitenden Daten vor, was zu Antworten führt. Beim Machine Learning geben Menschen sowohl die Daten als auch die dazugehörigen Antworten vor, und heraus kommen die Regeln. Diese Regeln sind dann auf neue Daten anwendbar und liefern eigenständige Antworten.


In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN01/Bilder/ML_Paradigma.png"
display(Image(url=url))

# Artificial Neural Networks (ANNs)

Künstliche Neuronale Netze (ANNs=artificial neural netwoks) sind die Kernkomponente des Deep Learning. Sie sind flexibel, mächtig und skalierbar, was sie ideal für große und hochgradig komplexe Machine-Learning-Aufgaben einsetzbar macht, wie beispielsweise die Klassifizierung von Milliarden Bildern (Google Images), Spracherkennung (Apples Siri), Videoempfehlungen für
Hunderte Millionen Nutzer pro Tag (YouTube) oder den Weltmeister im Brettspiel Go durch die Analyse von Millionen Partien und anschließendes Spielen gegen sich selbst zu schlagen, z.B. [AlphaGo von DeepMind](https://www.youtube.com/watch?v=HT-UZkiOLv8).

In diesem Kapitel werden wir künstliche neuronale Netze kennenlernen. Wir beginnen mit einem kurzen Überblick der ersten Architekturen von ANNs. Dann werden wir mehrschichtige Perzeptrons (MLPs) vorstellen und eines mithilfe von TensorFlow implementieren, um die Klassifikation der MNIST-Ziffern anzugehen.



## Was ist Deep Learning ?

Deep Learning ist ein Teilgebiet des Machine Learnings: ein neuer Ansatz, die Repräsentationen anhand von Daten zu erkennen, der den Schwerpunkt auf das Erlernen aufeinanderfolgender Layer (Schichten) mit zunehmend sinnvolleren Repräsentationen legt. Das Deep in Deep Learning bezieht sich also nicht auf irgendein tiefer gehendes durch diesen Ansatz erzielbares Verständnis, sondern steht für das Konzept aufeinanderfolgender Repräsentations-Layer. 

Die Anzahl der zu einem Datenmodell beitragenden Layer wird als die Tiefe des Modells bezeichnet.
Man hätte Deep Learning auch als Lernen durch schichtweise Repräsentationen oder Lernen durch hierarchische Repräsentationen bezeichnen können. Deep Learning umfasst heutzutage oft Dutzende oder sogar Hunderte aufeinanderfolgender Repräsentations-Layer – die alle durch die Bereitstellung der Trainingsdaten automatisch erlernt werden. Andere Ansätze des Machine Learnings konzentrieren sich tendenziell auf nur einen oder zwei Repräsentations-Layer und werden deshalb mitunter als Shallow Learning (»flaches« Lernen) bezeichnet.




### Was Deep Learning heute leistet:

- *Bildklassifizierung* auf nahezu menschlichem Niveau
- *Spracherkennung* auf nahezu menschlichem Niveau
- *Handschriftenerkennung* auf nahezu menschlichem Niveau
- Verbesserung der *Übersetzung von Fremdsprachen*
- Verbesserung der *Sprachsynthese*
- *Digitale Assistenten* wie Google Now oder Amazon Alexa
- *Selbstfahrende Autos* auf nahezu menschlichem Niveau
- Verbesserung *gezielter Werbung*, wie sie Google, Baidu und Bing einsetzen
- Verbesserung der *Suchergebnisse im Web*
- Beantwortung von in natürlicher Sprache gestellten Fragen
- Ein Programm schlägt den besten menschlichen Go-Spieler

### Von biologischen und künstlichen Neuronen 

NNs sind aus buchstäblich übereinandergestapelten Layern aufgebaut. 
- Der Begriff entstammt zwar der Neurobiologie, aber obwohl einige der grundlegenden Konzepte des Deep Learnings zum Teil durch unser Verständnis vom Gehirn inspiriert wurden, sind **Deep-Learning-Modelle keine Nachbildungen des Gehirns.**
- Es gibt keinerlei Hinweise darauf, dass das Gehirn irgendwelche Verfahren einsetzt, die den in modernen Deep-Learning-Modellen eingesetzten Lernmechanismen ähneln. 
- In populärwissenschaftlichen Artikeln wird gelegentlich behauptet, Deep Learning funktioniere wie das Gehirn oder sei dem Gehirn nachgebildet worden, **aber das stimmt nicht. Deep Learning hat nichts mit Neurobiologie zu tun.** 
- **Vergessen Sie die Mythen und Rätsel, die um die Vorstellung »genau wie unser Gehirn« gesponnen wurden, und am besten auch alles, was Sie über hypothetische Zusammenhänge zwischen Deep Learning und Biologie gelesen haben.**

### Entmystifizierung des kurzfristigen Hypes

Deep Learning hat zwar in den letzten Jahren bemerkenswerte Fortschritte erzielt, allerdings sind die Erwartungen in Bezug darauf, was auf diesem Gebiet im kommenden Jahrzehnt erreicht werden kann, viel zu hoch gesteckt. 
- Auch wenn weltbewegende Anwendungen wie selbstfahrende Autos schon in greifbarer Nähe sind, werden viele andere wahrscheinlich noch lange unerreichbar bleiben, wie etwa glaubwürdige Sprachdialogsysteme, Übersetzungen zwischen beliebigen Sprachen oder das Verständnis natürlicher Sprache auf menschlichem Niveau. 
- Vor allem die Berichte über allgemeine Intelligenz auf menschlichem Niveau sollten nicht allzu ernst genommen werden. Hohe Erwartungen, die kurzfristig nicht erfüllt werden, weil die Technologie noch nicht so weit ist, bringen das Risiko mit sich, dass weniger in die Forschung investiert wird und sich der Fortschritt auf diese Weise nachhaltig verlangsamt.

### Das Besondere an Deep Learning: Merkmalserstellung

- Der Hauptgrund dafür, dass sich Deep Learning so schnell durchgesetzt hat, ist die verbesserte Leistung, die es bei vielen Aufgabenstellungen bietet. Das ist jedoch nicht der einzige Grund. Deep Learning vereinfacht das Lösen von Aufgaben so sehr, weil es einen der entscheidenden Schritte des Machine-Learning-Workflows automatisiert: die **Merkmalserstellung**.

- Beim Deep Learning ist entscheidend, dass es dem Modell ermöglicht wird, alle Repräsentations-Layer zusammen und gleichzeitig zu erlernen (man bezeichnet das als **greedy**, also gierig) anstatt der Reihe nach. Durch dieses gleichzeitige Erlernen der Merkmale werden bei der Anpassung des Modells an eins der internen Merkmale alle anderen davon abhängigen Merkmale ebenfalls automatisch angepasst, ohne dass ein menschliches Eingreifen nötig wäre. 
- Alles wird durch **ein einziges Feedback-Signal** gesteuert: Alle Änderungen am Modell tragen zum eigentlichen Ziel bei. Dieses Verfahren ist sehr viel leistungsfähiger als ein massenhaftes Aufreihen von Shallow-Learning-Modellen, weil es ermöglicht, **komplexe, abstrakte Repräsentationen** zu erlernen, indem sie in eine lange Reihe dazwischenliegender Räume (Layer) unterteilt werden: Jeder dieser Räume unterscheidet sich von dem vorhergehenden nur durch eine einfache Transformation.
- Die Machine-Learning-Wettbewerbe bei [Kaggle](https://www.kaggle.com/) näher zu betrachten, stellt eine hervorragende Möglichkeit dar, sich ein Bild vom aktuellen Stand der Machine-Learning-Algorithmen und der verfügbaren Tools zu machen


Es gibt drei allgemeine Faktoren, die den Fortschritt beim Machine Learning
beeinflussen:
1. **Hardware:** Faktisch subventionierte der *Spielemarkt* das Supercomputing der nächsten Generation von KI-Anwendungen. Manchmal wird aus einem Spiel tatsächlich eine bedeutende Entwicklung. Eine NVIDIA TITAN X, eine Grafikkarte für Spiele, die
Ende 2015 1.000 Dollar kostete, kann bis zu 6,6 TFLOPS liefern (einfache Genauigkeit): 6,6 Billionen float32-Operationen pro Sekunde. Das ist etwa das 350-Fache dessen, was ein moderner Laptop leistet.
2. **Datenmengen und Benchmarks:** Wenn man eine Datenmenge nennen sollte, die als Katalysator für den Erfolg des Deep Learnings gedient hat, dann die *ImageNet-Datenbank*, die mehr als 1,4 Millionen Bilder enthält, die von Hand einer von 1.000 Kategorien zugewiesen worden sind (eine Kategorie pro Bild). Das Besondere an der ImageNet-Datenbank ist jedoch nicht nur ihre Größe, sondern auch der dazugehörige alljährlich stattfindende Wettbewerb.
3. **Verbesserte Algorithmen:** Erst nachdem folgende Verbesserungen umgesetzt wurden, war es möglich, Modelle mit zehn oder mehr Layern zu trainieren und die Vorteile des Deep Learnings zu nutzen:
    - verbesserte *Aktivierungsfunktionen* für die Layer
    - verbesserte Verfahren zur *Initialisierung der Gewichtungen* (He, Xavier), zunächst durch schichtweises Vortrainieren, was aber bald wieder aufgegeben wurde 
    - verbesserte *Optimierungsverfahren*, wie etwa RMSProp und Adam



# Historische Entwicklung
## McCulloch-Pitts-Zelle

Überraschenderweise gibt es ANNs schon eine ganze Weile: Das erste Mal wurden Sie 1943 vom Neurophysiologen **Warren McCulloch** und vom Mathematiker **Walter Pitts** erwähnt. In ihrem wegweisenden Artikel (https://goo.gl/Ul4mxW) »*A Logical Calculus of Ideas Immanent in Nervous Activity*« stellen McCulloch und Pitts ein vereinfachtes rechnerisches Modell vor, nach dem biologische Neuronen im Gehirn von Tieren zusammenarbeiten könnten, um komplexe Berechnungen mithilfe von Aussagenlogik durchzuführen. Dies war die erste Architektur eines künstlichen neuronalen Netzwerks. Seitdem wurden viele weitere Architekturen erfunden, wie wir noch sehen werden.

Die frühen Erfolge von ANNs bis zu den 1960ern führten zur verbreiteten Annahme, dass wir uns schon bald mit wirklich intelligenten Maschinen unterhalten würden. Als klar wurde, dass dieses Versprechen nicht eingelöst werden würde (zumindest
für eine lange Zeit), flossen Forschungsgelder in andere Richtungen, und für ANNs brach ein langes, dunkles Zeitalter an.

- [Wang: On the Origin of Deep Learning](https://arxiv.org/pdf/1702.07800.pdf)
- [Tappert: Who ist the father of Deep Learning?](https://ieeexplore.ieee.org/abstract/document/9070967)





## Das Perzeptron

Das 1957 von [**Frank Rosenblatt**](https://ieeexplore.ieee.org/abstract/document/9070967) erfundene [Perzeptron](https://de.wikipedia.org/wiki/Perzeptron) gehört zu den einfachsten Architekturen neuronaler Netze. Es baut auf einem leicht unterschiedlichen künstlichen Neuron auf, der **Linear Threshold Unit (LTU)**:
Die Ein- und Ausgaben sind nun Zahlen (anstatt binäre Ein-/Aus-Werte), und zu jeder Eingabeleitung gehört ein Gewicht. Die LTU berechnet eine gewichtete Summe ihrer Eingaben

$$ z = w_1 x_1 + w_2 x_2 + \dots + w_n x_n = \mathbf{w}^T  \mathbf{x}$$

und wendet dann eine *Aktivierungsfunktion* $\sigma(z)$ auf diese Summe an und gibt das Ergebnis aus: 

$$ h_w(x) = \sigma(\mathbf{w}^T  \mathbf{x})$$



In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN01/Bilder/perceptron.png"
display(Image(url=url))

Die im Perzeptron am häufigsten verwendete Aktivierungsfunktion ist die Heaviside-Funktion $\theta(z)$. Manchmal wird stattdessen die Vorzeichenfunktion $\operatorname{sgn}(z)$ verwendet.

$$
\begin{split}
\theta(z) =
\begin{cases}
0 & \text{if }z < 0\\
1 & \text{if }z \ge 0
\end{cases} & \quad\quad
\operatorname{sgn}(z) =
\begin{cases}
-1 & \text{if }z < 0\\
0 & \text{if }z = 0\\
+1 & \text{if }z > 0
\end{cases}
\end{split}
$$


Eine einzelne LTU lässt sich für einfache lineare binäre Klassifikationsaufgaben einsetzen. Sie berechnet eine Linearkombination der Eingabewerte und gibt die positive Kategorie aus, falls das Ergebnis einen Schwellenwert überschreitet, und andernfalls die negative Kategorie (wie bei der logistischen Regression oder einer linearen SVM). Die Entscheidungsgrenze stellt eine lineare Hyperebene dar, welche durch die Gewichte $\mathbf{w}$ definiert ist.

- Ein Perzeptron besteht einfach aus einer einzelnen Schicht LTUs, wobei jedes Neuron mit allen Eingabewerten verbunden ist.
- Diese Verbindungen werden oft als *spezielle Neuronen* zum Durchreichen der Information der *Eingabeneuronen* repräsentiert.
- Diese geben einfach nur die erhaltenen Eingabedaten aus. 
- Ausserdem wird üblicherweise ein zusätzliches *Bias-Merkmal* hinzugefügt ($x_0 = 1$). Dieses Bias-Merkmal wird durch ein weiteres spezielles Neuron repräsentiert, dem Bias-Neuron, das stets 1 ausgibt.

In der folgenden Abbildung ist ein Perzeptron mit zwei Eingaben und drei Ausgaben dargestellt. Dieses Perzeptron kann Datenpunkte gleichzeitig drei unterschiedlichen binären Kategorien zuordnen, wodurch es zu einem Klassifikator mit mehreren
Ausgaben wird.

In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN01/Bilder/perceptron2.png"
display(Image(url=url))

### Wie lässt sich ein Perzeptron trainieren? 

Der von Frank Rosenblatt vorgeschlagene Algorithmus zum Trainieren von Perzeptrons ist in weiten Teilen von der
**Hebbschen Lernregel** inspiriert. In seinem 1949 veröffentlichten Buch "*The Organization
of Behavior*" stellte Donald Hebb eine These vor, nach der die Verbindung zweier biologischer Neuronen stärker werde, wenn eines das andere häufig aktiviert.

Siegrid Löwel hat diesen Gedanken später als knackigen Satz formuliert: **»Cells that fire together, wire together.«** Dieser Grundsatz wurde später als Hebbsche Lernregel (oder Hebbsches Lernen) bekannt; dabei wird die Verbindung zweier
Neuronen immer dann verstärkt, wenn beide die gleiche Ausgabe haben. Perzeptrons lassen sich mit einer Variante dieser Regel trainieren, die die vom Netz begangenen Fehler berücksichtigt; Verbindungen, die zu einer falschen Ausgabe führen,
werden nicht verstärkt. Beim Trainieren wird dem Perzeptron ein einzelner Trainingsdatenpunkt vorgestellt und eine Vorhersage getroffen. Bei jedem Ausgabeneuron das eine falsche Vorhersage trifft, werden die Gewichte der Verbindungen verstärkt, die zu einer korrekten Vorhersage beigetragen hätten. 

$$ {w_{i,j}}^{(\text{next step})} = w_{i,j} + \eta \cdot (y_j - \hat{y}_j) x_i $$


Die Entscheidungsgrenze jedes Ausgabeneurons ist linear. Damit sind Perzeptrons nicht in der Lage, komplexe Muster zu erlernen (wie auf logistischer Regression aufbauende Klassifikatoren). Wenn sich die Trainingsdatenpunkte aber linear separieren
lassen, konvergiert dieser Algorithmus laut Rosenblatt zu einer Lösung. Dies nennt man das *Perzeptron-Konvergenztheorem*.

Scikit-Learn enthält die Klasse Perceptron, die ein einzelnes LTU-Netz implementiert.
Diese funktioniert wie erwartet – beispielsweise auf dem Iris-Datensatz.

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target


In [ ]:
y

## Explorative Datenanalyse (EDA)

- Bevor wir mit dem Trainieren starten, ist es *Pflicht*, die Daten statistisch zu untersuchen und zu visualisieren.
- Wir gehen dabei unvoreingenommen vor, wie ein Forscher oder Detektiv, der in den Daten spannende Zusammenhänge, Korrelationen, Kausaliäten oder Unstimmigkeiten zu *entdecken* sucht.

Um Daten zu sichten und visualisieren eignen sich die Pakete `seaborn`und `pandas`.

- Der Lilienblüten-Datensatz oder Fisher's Iris-Datensatz ist ein multivariater Datensatz, der von dem britischen Statistiker und Biologen Ronald Fisher in seinem 1936 erschienenen Aufsatz The use of multiple measurements in taxonomic problems (Die Verwendung von Mehrfachmessungen bei taxonomischen Problemen) als Beispiel für eine lineare Diskriminanzanalyse verwendet und berühmt gemacht wurde. 
- Der Datensatz besteht aus 50 Proben von jeder der drei Irisarten (Iris setosa, Iris virginica und Iris versicolor). 
- Bei jeder Probe wurden vier Merkmale gemessen: die Länge und die Breite der Kelch- und Blütenblätter in Zentimetern.
- Auf der Grundlage der Kombination dieser vier Merkmale entwickelte Fisher ein *lineares Diskriminanzmodell* zur Unterscheidung der Arten voneinander. 

- Iris Setosa
- Iris Versicolor
- Iris Virginica


Es handelt sich hier um eine Aufgabe des **überwachten Lernens** (supervised): **Klassifizierung**
- Die *Features* $X_i \, (i=1, \dots 4)$ sind die Längen der Blüten und Kelchblätter in cm.
- Das *Label* $y$ ist die Art der Lilie (setosa, ,virginica, versicolor), hier als Integer (diskret) repräsentiert.
- Wäre das Label als Zeichenkette codiert (d.h. mit Namen), müssten wir erst durch einen `LabelEncoder` diese Klassen erzeugen.

Wir erzeugen als erstes einen `pandas`-Dataframe (tidy data) dieses Datensatzes.

In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN01/Bilder/iris_setosa.jpg"
display(Image(url=url))
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN01/Bilder/iris_versicolor.jpg"
display(Image(url=url))
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN01/Bilder/iris_virginica.jpg"
display(Image(url=url))

In [ ]:
import pandas as pd
import seaborn as sns

iris.feature_names
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df["class"] = y
df.tail(10)
df.dropna()

Mit `describe`lässt sich die 5-Werte-Statistik berechnen.

In [ ]:
df.describe()

Spannend sind immer die Korrelationen zwichen den Features und den Features und der Response.

In [ ]:
df.corr()

In [ ]:
# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(8, 7))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=0.3,
    center=0,
    annot=True,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
)

Am besten visualisiert man diese durch einen `pairplot` (Scatterplot, Streudiagramm):

In [ ]:
sns.pairplot(df)

In [ ]:
sns.boxplot(data=df)
plt.xticks(rotation=45);

## Binäre Klassifizierung

- Nun verwenden wir nur zwei Merkmale (Featrures) als Input (petal length, petal width) und entscheiden nur, ob es sich um eine Iris Setosa handelt oder nicht.
- Wir verwenden das Perceptron-Modell von scikit-learn.

In [ ]:
X = iris.data[:, (2, 3)]  # petal length, petal width
y = (iris.target == 0).astype(int)

per_clf = Perceptron(random_state=42)
per_clf.fit(X, y)

y_pred = per_clf.predict([[4.5, 0.8]])
y_pred

In [ ]:
per_clf = Perceptron(random_state=42)
per_clf.fit(X, y)

y_pred = per_clf.predict([[2, 0.5]])

In [ ]:
a = -per_clf.coef_[0][0] / per_clf.coef_[0][1]
b = -per_clf.intercept_ / per_clf.coef_[0][1]

axes = [0, 5, 0, 2]

x0, x1 = np.meshgrid(
    np.linspace(axes[0], axes[1], 500).reshape(-1, 1),
    np.linspace(axes[2], axes[3], 200).reshape(-1, 1),
)
X_new = np.c_[x0.ravel(), x1.ravel()]
y_predict = per_clf.predict(X_new)
zz = y_predict.reshape(x0.shape)

In [ ]:
y_pred

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(X[y == 0, 0], X[y == 0, 1], "bs", label="Not Iris-Setosa")
plt.plot(X[y == 1, 0], X[y == 1, 1], "yo", label="Iris-Setosa")

plt.plot([axes[0], axes[1]], [a * axes[0] + b, a * axes[1] + b], "k-", linewidth=3)


custom_cmap = ListedColormap(["#9898ff", "#fafab0"])

plt.contourf(x0, x1, zz, cmap=custom_cmap)
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(loc="lower right", fontsize=14)
plt.axis(axes)

plt.savefig("Bilder/perceptron_iris_plot.pdf")
plt.show()

In einer Monografie aus dem Jahr 1969 mit dem Titel *Perceptrons* hoben **Marvin Minsky** und **Seymour Papert** eine Reihe ernster Nachteile von Perzeptrons hervor, insbesondere ihr Versagen bei einer Reihe trivialer Probleme (z.B. das exklusive OR
(XOR) als Klassifikationsaufgabe; dargestellt auf der linken Seite der folgenden Abbildung. Natürlich gilt dies auch für jedes andere lineare Klassifikationsmodell (wie die logistische Regression). In der Folge wandten sich viele Wissenschaftler vom *Konnektionismus* insgesamt ab (d.h. dem Studium neuronaler Netze), um sich übergeordneten Aufgabenstellungen wie
Logik, Problemlösung und Suche zuzuwenden. Dies läutete den **ersten AI-Winter** ein.

In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN01/Bilder/perceptron3.png"
display(Image(url=url))

Es stellt sich aber heraus, dass sich einige dieser Beschränkungen aufheben lassen, indem man mehrere Perzeptrons in Reihe schaltet. Das dabei entstehende ANN bezeichnet man als mehrschichtiges Perzeptron (MLP).

## Mehrschichtiges Perzeptron (MLP) und Backpropagation

Ein MLP setzt sich aus einer Eingabeschicht (zum Durchreichen) und einer oder mehreren Schichten von LTUs zusammen, den verborgenen Schichten, und einer letzten Schicht LTUs, der Ausgabeschicht. Bis auf die Ausgabeschicht
enthält jede Schicht ein Bias-Neuron und ist mit der nächsten Schicht vollständig verbunden. Wenn ein ANN aus zwei oder mehr verborgenen Schichten besteht, bezeichnet man es auch als Deep-Learning-Netz (DNN).

In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN01/Bilder/MLP.png"
display(Image(url=url))

Viele Jahre lang haben Forscher vergeblich nach einer Möglichkeit zum Trainieren von MLPs gesucht. Im Jahr 1986 aber publizierten D. E. Rumelhart et al. einen wegweisenden Artikel (https://goo.gl/Wl7Xyc), der den Backpropagation-Algorithmus
bekannt machte. Heute würden wir ihn als **Gradientenverfahren mit Autodiff im Reverse-Modus** beschreiben.

- **Vorwärtspfad**: Der Algorithmus speist jeden Trainingsdatenpunkt in das Netz ein und berechnet die Ausgabe jedes Neurons in jeder aufeinanderfolgenden Schicht (dies ist ein vorwärts gerichteter Durchlauf wie beim Treffen von Vorhersagen). 
- **Error-Berechnung**: Anschliessend wird der Fehler der Ausgabe des Netzes gemessen (d.h. die Differenz zwischen der gewünschten und der tatsächlichen Ausgabe). Für jedes Neuron in der letzten verborgenen Schicht wird dann bestimmt, wie stark es zum Fehler der Ausgabe beitrug.
- **Rückwärtspfad**:Anschliessend wird berechnet, welcher Teil dieser Fehlerbeiträge auf jedes Neuron in der vorigen verborgenen Schicht entfiel – und so weiter, bis der Algorithmus die Eingabeschicht erreicht. In diesem rückwärtigen Durchlauf wird der Fehlergradient über sämtliche Gewichte im Netz zurückverfolgt (daher der Name Backpropagation). 

Der letzte Schritt des Backpropagation-Algorithmus ist ein Schritt im Gradientenverfahren auf allen Gewichten im Netz unter Verwendung des zuvor bestimmten Gradienten.

# Aktivierungsfunktionen

Damit dieser Algorithmus gut funktioniert, nahmen die Autoren eine wichtige Änderung an der Architektur des MLP vor: Sie ersetzten die Stufenfunktion mit der logistischen Funktion 

$$ 
\sigma(z) = \frac{1}{1 + e^{–z}}
$$. 

Dies erwies sich als entscheidend, weil die Stufenfunktion nur flache Abschnitte enthält und es daher keinen Gradienten
gibt (die Gradientenmethode kann sich auf einer flachen Oberfläche nicht bewegen), wohingegen die Ableitung der logistischen Funktion überall ungleich null ist. Damit kann die Gradientenmethode an jeder Stelle ein wenig voranschreiten.
Der Backpropagation-Algorithmus lässt sich auch mit anderen Aktivierungsfunktionen als der logistischen Funktion einsetzen. 


Zwei weitere **beliebte Aktivierungsfunktionen** sind:


- **Der Tangens hyperbolicus** $\tanh(z)$: Wie die logistische Funktion ist der Tangens S-förmig, stetig und differenzierbar, aber die Ausgabewerte liegen im Bereich zwischen –1 und 1 (anstatt von 0 bis 1 bei der logistischen Funktion). Damit wird die Ausgabe jeder Schicht zu Beginn des Trainings tendenziell normalisiert (d.h. auf 0 zentriert). Dies beschleunigt bisweilen die Konvergenz.
- Die **ReLU-Funktion** $\operatorname{ReLU}(z) = \max (0, z)$: Diese Funktion ist stetig, aber bei $z = 0$ leider nicht differenzierbar. In der Praxis funktioniert diese Funktion aber sehr gut und ist ausserdem schnell berechenbar. Wichtiger ist, dass sie keinen maximalen Ausgabewert besitzt, was einige Probleme des Gradientenverfahrens umgeht.

In [ ]:
def logit(z):
    return 1 / (1 + np.exp(-z))


def relu(z):
    return np.maximum(0, z)


def derivative(f, z, eps=0.000001):
    return (f(z + eps) - f(z - eps)) / (2 * eps)

In [ ]:
z = np.linspace(-5, 5, 200)

plt.figure(figsize=(11, 4))

plt.subplot(121)
plt.plot(z, np.sign(z), "r-", linewidth=2, label="Step")
plt.plot(z, logit(z), "g--", linewidth=2, label="Logit")
plt.plot(z, np.tanh(z), "b-", linewidth=2, label="Tanh")
plt.plot(z, relu(z), "m-.", linewidth=2, label="ReLU")
plt.grid(True)
plt.legend(loc="center right", fontsize=14)
plt.title("Aktivierungsfunktionen", fontsize=14)
plt.axis([-5, 5, -1.2, 1.2])

plt.subplot(122)
plt.plot(z, derivative(np.sign, z), "r-", linewidth=2, label="Step")
plt.plot(0, 0, "ro", markersize=5)
plt.plot(0, 0, "rx", markersize=10)
plt.plot(z, derivative(logit, z), "g--", linewidth=2, label="Logit")
plt.plot(z, derivative(np.tanh, z), "b-", linewidth=2, label="Tanh")
plt.plot(z, derivative(relu, z), "m-.", linewidth=2, label="ReLU")
plt.grid(True)
# plt.legend(loc="center right", fontsize=14)
plt.title("Ableitung", fontsize=14)
plt.axis([-5, 5, -0.2, 1.2])

plt.savefig("activation_functions_plot.pdf")
plt.show()

### Klassifizieren mit einem softmax-Layer

Ein MLP wird häufig zur Klassifikation eingesetzt, wobei jede Ausgabe einer anderen binären Kategorie entspricht (z.B. Spam/Ham, dringend/nicht dringend und so weiter). Falls die Kategorien sich gegenseitig ausschliessen (z.B. Kategorien von 0
bis 9 bei der Klassifikation der Bilder von Ziffern), werden in der Ausgabeschicht normalerweise die einzelnen Aktivierungsfunktionen durch eine gemeinsame Softmax-Funktion ersetzt. Die **Softmax-Funktion** ist definiert durch:

$$
\hat{p}_k = \sigma\left[\mathbf{s}(\mathbf{x})\right]_k = \dfrac{\exp\left(s_k(\mathbf{x})\right)}{\sum\limits_{j=1}^{K}{\exp\left(s_j(\mathbf{x})\right)}}
$$

Die Ausgabe jedes Neurons entspricht dann der geschätzten Wahrscheinlichkeit der entsprechenden Kategorie. Beachten Sie, dass das Signal nur in eine Richtung fliesst (von der Ein- zur Ausgabe). Daher ist diese Architektur ein Beispiel für ein **Feed-Forward-Netz** (engl. Feedforward Neural Networks, FNN).


In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN01/Bilder/softmax.png"
display(Image(url=url))

Bemerkung: *Biologische Neuronen scheinen eine in etwa sigmoide (S-förmige) Aktivierungsfunktion zu implementieren, daher hatte sich die Wissenschaft sehr lange auf Sigmoidalfunktionen eingeschossen. In ANNs funktioniert ReLU als Aktivierungsfunktion jedoch meist besser. In diesem Fall war die biologische Analogie irreführend.*

python 3.8.6/tensorflow==2.5.0/keras==2.4.3

Die MNIST-Datensammlung ist Bestandteil von `torchvision` und steht in Form von vier
Numpy-Arrays zur Verfügung.

In [ ]:
# Define the transformation for the MNIST dataset
transform = transforms.ToTensor()

# Load the MNIST dataset
train_dataset = datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)

# Extract the data and labels
train_images = train_dataset.data
train_labels = train_dataset.targets
test_images = test_dataset.data
test_labels = test_dataset.targets

# Check the shape of the training images
print(train_images.shape)

Wenn Sie sich eingehender mit Machine Learning befassen, wird Ihnen die MNIST-Datensammlung immer wieder begegnen, in wissenschaftlichen Arbeiten, Blogbeiträgen usw. Der folgende Code-Snippet zeigt einige Beispiele.

In [ ]:
def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap=matplotlib.cm.binary, interpolation="nearest")
    plt.axis("off")

In [ ]:
plot_digit(train_images[1549])

In [ ]:
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size, size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap=matplotlib.cm.binary, **options)
    plt.axis("off")

In [ ]:
fig = plt.figure(figsize=(9, 9))
example_images = train_images[:40000:600]
plot_digits(example_images, images_per_row=10)
plt.show()
# fig.savefig('more_digits_plot.png',format='png',dpi=1200)

`train_images` und `train_labels` bilden den "Trainingssatz", die Daten, aus denen das Modell lernt. Das Modell wird dann auf der
"test set", `test_images` und `test_labels`. Unsere Bilder sind als Numpy-Arrays codiert und die Labels sind einfach ein Array von Ziffern, die von 0 bis 9 reichen. Es besteht eine Eins-zu-Eins-Entsprechung zwischen den Bildern und den Etiketten.

Schauen wir uns die Trainingsdaten an:

In [ ]:
train_images.shape

In [ ]:
len(train_labels)

In [ ]:
train_labels

Schauen wir uns die Testdaten an:

In [ ]:
test_images.shape

In [ ]:
len(test_labels)

In [ ]:
test_labels

Unser Arbeitsablauf sieht wie folgt aus: Zuerst präsentieren wir unser neuronales Netz mit den Trainingsdaten `train_images` und `train_labels`. Das
Netzwerk lernt dann, Bilder und Labels zuzuordnen. Schließlich werden wir das Netzwerk bitten, Vorhersagen für `test_images` zu erstellen, und wir überprüfen, ob diese Vorhersagen mit den Labels von `test_labels` übereinstimmen.

Lassen Sie uns unser Netzwerk aufbauen – denken Sie noch einmal daran, dass Sie noch nicht alles über dieses Beispiel verstehen sollen.


Der folgende Code implementiert ein einfaches neuronales Netz zur Klassifikation von MNIST-Daten in PyTorch Lightning. Hier ist eine ausführliche Erklärung der einzelnen Schritte:


#### **1. Bibliotheken importieren**

```python
import torch
from torch import nn
import pytorch_lightning as pl
```

- **`torch`**: Hauptbibliothek für maschinelles Lernen mit PyTorch.
- **`nn`**: Enthält Module, um neuronale Netzwerke aufzubauen (z. B. `Linear`, `ReLU`).
- **`pytorch_lightning`**: Eine High-Level-Bibliothek, die das Training und die Verwaltung von Modellen in PyTorch vereinfacht.



#### **2. Definition des Modells**

```python
class MNISTModel(pl.LightningModule):
    def __init__(self, input_size=28*28, hidden_size=512, num_classes=10, learning_rate=0.001):
        super(MNISTModel, self).__init__()
        self.save_hyperparameters()
        
        # Define the model architecture
        self.model = nn.Sequential(
            nn.Linear(self.hparams.input_size, self.hparams.hidden_size),
            nn.ReLU(),
            nn.Linear(self.hparams.hidden_size, self.hparams.num_classes),
            nn.Softmax(dim=1)
        )
        
        # Define the loss function
        self.loss_fn = nn.CrossEntropyLoss()
```

##### **Erklärung:**
- **`__init__`**: Initialisiert das Modell.
    - `input_size`: Größe des Eingabevektors (28 × 28 für MNIST-Bilder).
    - `hidden_size`: Anzahl der Neuronen in der versteckten Schicht.
    - `num_classes`: Anzahl der Klassen (10 Ziffern: 0–9).
    - `learning_rate`: Lernrate für den Optimierungsalgorithmus.
- **`save_hyperparameters()`**: Speichert alle Hyperparameter, damit sie für Logging und Checkpoints verfügbar sind.
- **Modellarchitektur (`nn.Sequential`)**:
    - `nn.Linear`: Vollverbundene Schichten (Input → Hidden → Output).
    - `nn.ReLU`: Aktivierungsfunktion, um Nichtlinearitäten einzuführen.
    - `nn.Softmax`: Normalisiert die Ausgaben der letzten Schicht zu Wahrscheinlichkeiten.
- **Verlustfunktion (`CrossEntropyLoss`)**:
    - Misst den Unterschied zwischen vorhergesagten Wahrscheinlichkeiten und den Zielwerten.



#### **3. Vorwärtspropagation**

```python
def forward(self, x):
    return self.model(x)
```

- **`forward`**: Definiert, wie die Eingabedaten durch das Modell verarbeitet werden.
- Nimmt den Eingabevektor `x` und leitet ihn durch die Modellarchitektur (`self.model`).



#### **4. Trainingsschritt**

```python
def training_step(self, batch, batch_idx):
    x, y = batch
    x = x.view(x.size(0), -1)  # Flatten the input
    preds = self(x)
    loss = self.loss_fn(preds, y)
    self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    return loss
```

##### **Erklärung:**
- **`training_step`**: Führt einen einzelnen Trainingsschritt aus.
    - `batch`: Ein Batch von Eingabedaten (Bilder und Labels).
    - `batch_idx`: Index des Batches.
- **Input-Transformation**:
    - `x.view(x.size(0), -1)`: Flacht die Bilder (28 × 28) zu einem Vektor (784) ab.
- **Vorhersage**:
    - `preds = self(x)`: Berechnet die Ausgabe des Modells.
- **Verlustberechnung**:
    - `self.loss_fn(preds, y)`: Vergleicht die Vorhersagen mit den Labels und berechnet den Cross-Entropy-Verlust.
- **Logging**:
    - `self.log`: Loggt den Trainingsverlust für jeden Schritt und jede Epoche (für Monitoring).





In [ ]:
# pip install lightning


class MNISTModel(pl.LightningModule):
    def __init__(
        self, input_size=28 * 28, hidden_size=512, num_classes=10, learning_rate=0.001
    ):
        super(MNISTModel, self).__init__()
        self.save_hyperparameters()

        # Define the model architecture
        self.model = nn.Sequential(
            nn.Linear(self.hparams.input_size, self.hparams.hidden_size),
            nn.ReLU(),
            nn.Linear(self.hparams.hidden_size, self.hparams.num_classes),
            nn.Softmax(dim=1),
        )

        # Define the loss function
        self.loss_fn = nn.CrossEntropyLoss()
        self.train_accuracy = Accuracy(task="multiclass", num_classes=10)
        self.val_accuracy = Accuracy(task="multiclass", num_classes=10)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1)
        preds = self(x)
        loss = self.loss_fn(preds, y)

        # Log loss and accuracy
        self.log(
            "train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        self.log(
            "train_accuracy",
            self.train_accuracy(preds, y),
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1)
        preds = self(x)
        loss = self.loss_fn(preds, y)

        # Log validation loss and accuracy
        self.log(
            "val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        self.log(
            "val_accuracy",
            self.val_accuracy(preds, y),
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)


In [ ]:
# Select device: CUDA if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

# Move model to selected device
model = MNISTModel().to(device)


In [ ]:
summary(model, input_size=(1, 28 * 28))  # 1 is the batch size (arbitrary here)

Der wichtigste Baustein eines NNs ist layers, ein **Datenverarbeitungsmodul**, das Sie sich wie einen Datenfilter vorstellen können. Es nimmt Daten entgegen und gibt sie in einer nützlicheren Form wieder aus. Das Modul extrahiert aus den eingegebenen Daten Repräsentationen, die für eine gegebene Aufgabe hoffentlich sinnvoller sind als die Rohdaten. 

Ein Grossteil des Deep Learnings besteht aus dem Verketten einfacher Layer, die eine Art von schrittweiser Datendestillation implementieren. Ein Deep-Learning-Modell ähnelt einem *Sieb für die Datenverarbeitung*, das aus einer Abfolge von immer ausgeklügelteren Datenfiltern besteht – den Layern.

In diesem Fall besteht das NN aus zwei vollständig miteinander verbundenen Layern des Typs Dense. Beim zweiten (und letzten) Layer handelt es sich um einen 10-fachen Softmax-Layer. Das bedeutet, dass er ein Array ausgibt, das 10 Wahrscheinlichkeitswerte enthält (die sich zu 1 summieren). Diese Werte geben die Wahrscheinlichkeit an, dass die aktuelle Ziffer zu einer der 10 Ziffernklassen gehört.


In [ ]:
print(ModelSummary(model, max_depth=-1))  # max_depth=-1 prints all layers


Um das NN für das Training vorzubereiten, müssen wir bei der Kompilierung drei weitere Dinge festlegen:
1. Eine **Verlustfunktion (loss)** – Sie beschreibt, wie das NN seine Leistung für die Trainingsdaten beurteilen kann, und damit auch, wie es Korrekturen in der richtigen Richtung vornehmen kann.
2. Einen **Optimierer (optimizer)** – Der Mechanismus, durch den sich das NN anhand der bekannten Daten und der dazugehörigen Werte der Verlustfunktion selbst aktualisiert.
3. Beim Trainieren und Testen **zu überwachende Kennzahlen (metric)** – In diesem Fall sind wir lediglich an der Korrektklassifizierungsrate interessiert (dem Anteil der richtig klassifizierten Bilder).



#### **5. Optimierer konfigurieren**

```python
def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
```

- **`configure_optimizers`**: Definiert den Optimierungsalgorithmus.
    - **`Adam`**: Ein Optimierer, der den Gradientenabstieg verbessert.
    - Verwendet die gespeicherte Lernrate `self.hparams.learning_rate`.



#### **6. Training des Modells**

```python
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Load the dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Instantiate the model
model = MNISTModel()

# Train the model
trainer = pl.Trainer(max_epochs=5, log_every_n_steps=10)
trainer.fit(model, train_loader)
```

##### **Erklärung:**
1. **Datenvorbereitung**:
    - **`transforms.ToTensor()`**: Konvertiert Bilder in PyTorch-Tensoren.
    - **`transforms.Normalize`**: Normalisiert die Pixelwerte auf den Bereich [-1, 1].
2. **Datenladefunktion (`DataLoader`)**:
    - Erzeugt Batches der Eingabedaten (Batchgröße: 64).
    - `shuffle=True`: Mischt die Daten für jeden Durchgang (Epoche).
3. **Modellinstanziierung**:
    - `model = MNISTModel()`: Erstellt eine Instanz des definierten Modells.
4. **Trainer**:
    - `pl.Trainer`: Verwaltet das Training.
    - `max_epochs=5`: Trainiert das Modell für 5 Epochen.
    - `log_every_n_steps=10`: Loggt alle 10 Schritte den Fortschritt.


Vor dem Training werden wir unsere Daten vorverarbeiten, indem wir sie in die vom Netzwerk erwartete Form umformen und sie so skalieren, dass alle Werte im Intervall `[0, 1]` leigen. Die Trainingsbilder sind beispielsweise in einem Array der
Form `[60000, 28, 28]` und des Typs `uint8` mit Werten im Intervall `[0, 255]` gespeichert. Wir wandeln es in ein `float32`-Array der Form `[60000, 28 * 28]` mit Werten zwischen `0` und `1` um.

In [ ]:
# Load the dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)
train_dataset = datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
train_loader = DataLoader(
    train_dataset, batch_size=64, shuffle=True, num_workers=15, persistent_workers=True
)

In [ ]:
# Validation dataset
val_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
val_loader = DataLoader(
    val_dataset, batch_size=64, shuffle=False, num_workers=15, persistent_workers=True
)


Jetzt können wir das NN trainieren. Zu diesem Zweck wird in Keras die `fit()`- Methode aufgerufen, die das Modell an die Trainingsdaten anpasst:

In [ ]:
# Logger initialisieren
logger = CSVLogger(save_dir="logs", name="mnist")

# Trainer initialisieren
trainer = pl.Trainer(max_epochs=10, log_every_n_steps=5, logger=logger)

# Training starten
trainer.fit(model, train_loader, val_loader)


Während des Trainings werden zwei Werte angezeigt: 
- der **Wert der Verlustfunktion (loss)** und 
- die **Korrektklassifizierungsrate (acc)** für die Trainingsdaten.

Wir erzielen für die Trainingsdaten schon bald eine Korrektklassifizierungsrate von 0.989 (98.9 %). Nun überprüfen wir, ob das Modell mit den Testdaten genauso gut zurechtkommt:

In [ ]:
# results = trainer.validate(model, dataloaders=val_loader)
# print(results)  # This will output the logged validation metrics

Die Korrektklassifizierungsrate für die Testdatenmenge beträgt 97,8% – deutlich weniger als bei den Trainingsdaten. Diese Differenz zwischen den Korrektklassifizierungsraten von Trainings- und Testdatenmenge ist ein typisches Beispiel für
eine Überanpassung: Die Leistung von Machine-Learning-Modellen ist bei neuen Daten tendenziell schlechter als bei den Trainingsdaten. Die **Überanpassung (overfitting)** ist das Hauptthema einer der folgenden Lektionen.

In [ ]:
# CSV-Logdatei laden
log_data = pd.read_csv("./logs/mnist/version_0/metrics.csv")


log_data.tail()


In [ ]:
# Trainings- und Validierungsdaten extrahieren
train_loss = log_data["train_loss_epoch"].dropna()
val_loss = log_data["val_loss_epoch"].dropna()
train_acc = log_data["train_accuracy"].dropna()
val_acc = log_data["val_accuracy"].dropna()

In [ ]:
# Plot: Loss
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label="Train Loss", marker="o")
plt.plot(val_loss, label="Validation Loss", marker="o")
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()

# Plot: Accuracy
plt.figure(figsize=(10, 5))
plt.plot(train_acc, label="Train Accuracy", marker="o")
plt.plot(val_acc, label="Validation Accuracy", marker="o")
plt.title("Training and Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()
plt.show()